In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('coded.csv')

In [9]:
corr = df.corr(numeric_only=True)
corr.style.background_gradient(cmap='coolwarm')

,RespondentID,How often do you travel by plane?,Do you ever recline your seat when you fly?,Do you have any children under 18?,Who should have control over the window shade?,Is itrude to move to an unsold seat on a plane?,"Generally speaking, is it rude to say more than a few words tothe stranger sitting next to you on a plane?","On a 6 hour flight from NYC to LA, how many times is it acceptable to get up if you're not in an aisle seat?","Under normal circumstances, does a person who reclines their seat during a flight have any obligation to the person sitting behind them?",Is itrude to recline your seat on a plane?,"Given the opportunity, would you eliminate the possibility of reclining seats on planes entirely?",Is it rude to ask someone to switch seats with you in order to be closer to friends?,Is itrude to ask someone to switch seats with you in order to be closer to family?,Is it rude to wake a passenger up if you are trying to go to the bathroom?,Is itrude to wake a passenger up if you are trying to walk around?,"In general, is itrude to bring a baby on a plane?","In general, is it rude to knowingly bring unruly children on a plane?",Have you ever used personal electronics during take off or landing in violation of a flight attendant's direction?,Have you ever smoked a cigarette in an airplane bathroom when it was against the rules?,Gender,Household Income,Education,Location (Census Region)
RespondentID,1.000000,0.021625,-0.015843,-0.022741,-0.066849,0.020482,0.039158,0.104820,0.040150,-0.047809,-0.016874,0.102970,0.045258,-0.043971,-0.031457,0.015626,0.003829,-0.028416,0.005418,-0.261733,-0.067615,-0.008335,-0.031399
How often do you travel by plane?,0.021625,1.000000,-0.009314,-0.023420,0.022836,-0.057648,-0.042614,0.008884,0.056399,-0.105118,-0.058495,-0.019034,-0.056168,-0.052442,0.040365,-0.051300,-0.012186,-0.201084,-0.240017,0.038965,-0.264177,-0.221369,0.013471
Do you ever recline your seat when you fly?,-0.015843,-0.009314,1.000000,-0.046649,0.020249,0.067699,0.060408,0.080337,0.382798,0.515390,0.401736,0.038480,0.033391,0.007597,0.053686,-0.002774,0.002252,-0.121906,-0.058555,-0.015669,-0.017903,-0.012166,0.025230
Do you have any children under 18?,-0.022741,-0.023420,-0.046649,1.000000,0.016180,-0.023581,-0.053721,-0.004319,-0.024322,-0.073950,-0.068244,0.004090,-0.026655,-0.004470,-0.002302,-0.160285,-0.213576,-0.040712,0.045287,-0.023335,0.087101,0.023069,-0.012349
Who should have control over the window shade?,-0.066849,0.022836,0.020249,0.016180,1.000000,-0.022682,-0.042176,-0.077756,0.112846,-0.028250,-0.019704,-0.082179,-0.110448,-0.011895,-0.042005,-0.073334,-0.050744,-0.077919,-0.054504,0.029293,0.032968,0.052535,0.007274
Is itrude to move to an unsold seat on a plane?,0.020482,-0.057648,0.067699,-0.023581,-0.022682,1.000000,0.228318,0.071082,0.033922,0.128089,0.084070,0.240077,0.225459,0.099353,0.130140,0.097772,0.073564,-0.041826,0.085049,-0.047418,-0.104394,-0.122562,0.019454
"Generally speaking, is it rude to say more than a few words tothe stranger sitting next to you on a plane?",0.039158,-0.042614,0.060408,-0.053721,-0.042176,0.228318,1.000000,0.107464,0.031261,0.155134,0.119054,0.136300,0.144095,0.147326,0.122639,0.210942,0.104410,0.021499,0.060857,0.002694,-0.010676,0.005086,0.033809
"On a 6 hour flight from NYC to LA, how many times is it acceptable to get up if you're not in an aisle seat?",0.104820,0.008884,0.080337,-0.004319,-0.077756,0.071082,0.107464,1.000000,0.017505,0.078994,0.092688,0.134944,0.127673,0.173034,0.254337,0.032252,0.020783,0.018705,0.039229,0.067805,-0.032595,-0.058349,0.027404
"Under normal circumstances, does a person who reclines their seat during a flight have any obligation to the person sitting behind them?",0.040150,0.056399,0.382798,-0.024322,0.112846,0.033922,0.031261,0.017505,1.000000,0.373565,0.239914,-0.012058,-0.027205,0.046859,0.044675,-0.015835,0.027764,-0.103466,-0.012245,0.053964,-0.046892,-0.021801,0.026319
Is itrude to recline your seat on a plane?,-0.0